In [20]:
import cv2
from skimage import feature
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage.color import rgb2lab
import os
import cv2
import numpy as np
from skimage.feature import hog

In [21]:

class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius
    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist

In [23]:
class Colorize:
    def __init__(self):
        self.threshold=None
        self.dicto = {}
        self.example = 0
    def feature_extractor(self, img_L, img_A, img_B):
        # Check if the input image has the correct dimensions
        if img_L.shape[0] <= 8 or img_L.shape[1] <= 8:
            # Resize the image to meet the minimum size requirements for HOG computation
            img_L_resized = cv2.resize(img_L, (8, 8))  # Adjust the size as needed
            img_A_resized = cv2.resize(img_A, (8, 8))  # Resize corresponding channels if needed
            img_B_resized = cv2.resize(img_B, (8, 8))
            
            # Proceed with feature extraction on the resized image
            hog_features, hog_image = hog(img_L_resized, orientations=9, pixels_per_cell=(8, 8),
                                            cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')

            # local binary pattern
            lbp = LocalBinaryPatterns(24,8)
            lbp_features = lbp.describe(img_L_resized)

            # # histogram of oriented gradients
            # hist = cv2.calcHist([img_L_resized],[0],None,[256],[0,256]).flatten()
            # print(hist.shape)

            features = tuple(np.concatenate((hog_features, lbp_features)))
            self.dicto[features] = np.array([np.mean(img_A_resized), np.mean(img_B_resized)])
            self.example = features
            return features


        else:
            # Proceed with feature extraction on the original image
            hog_features, hog_image = hog(img_L, orientations=9, pixels_per_cell=(8, 8),
                                            cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')

            # local binary pattern
            lbp = LocalBinaryPatterns(24,8)
            lbp_features = lbp.describe(img_L)

            # # histogram of oriented gradients
            # hist = cv2.calcHist([img_L],[0],None,[256],[0,256]).flatten()
            # print(hist.shape)

            features = tuple(np.concatenate((hog_features, lbp_features)))
            self.dicto[features] = np.array([np.mean(img_A), np.mean(img_B)])
            
            
            return features

    
    def feature_extractor_target_image(self, target_img_L, target_img_A, target_img_B):
        # Check if the input image has the correct dimensions
        if target_img_L.shape[0] < 8 or target_img_L.shape[1] < 8:
            # Resize the image to meet the minimum size requirements for HOG computation
            target_img_L_resized = cv2.resize(target_img_L, (8, 8))  # Adjust the size as needed
            target_img_A_resized = cv2.resize(target_img_A, (8, 8))  # Resize corresponding channels if needed
            target_img_B_resized = cv2.resize(target_img_B, (8, 8))

            # Proceed with feature extraction on the resized image
            hog_features, hog_image = hog(target_img_L_resized, orientations=9, pixels_per_cell=(8, 8),
                                            cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')

            # Local binary pattern
            lbp = LocalBinaryPatterns(24, 8)
            lbp_features = lbp.describe(target_img_L_resized)

            # # Histogram of oriented gradients
            # hist = cv2.calcHist([target_img_L_resized], [0], None, [256], [0, 256]).flatten()

            features = np.concatenate((hog_features, lbp_features))
            return features

        else:
            target_img_L_resized = cv2.resize(target_img_L, (8, 8))
            # Proceed with feature extraction on the original image
            hog_features, hog_image = hog(target_img_L_resized, orientations=9, pixels_per_cell=(8, 8),
                                            cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')

            # Local binary pattern
            lbp = LocalBinaryPatterns(24, 8)
            lbp_features = lbp.describe(target_img_L)

            # # Histogram of oriented gradients
            # hist = cv2.calcHist([target_img_L], [0], None, [256], [0, 256]).flatten()

            features = np.concatenate((hog_features, lbp_features))
            return features

    
    def train(self,directory_path):
        window_size = 4
        for i,filename in enumerate(os.listdir(directory_path)):
            if i < 10:
                img_path = os.path.join(directory_path, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (64, 64))
                lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
                L_channel = lab_image[:, :, 0]

                    
                    
                print(L_channel.shape[0], L_channel.shape[1])
                for i in range(L_channel.shape[0] - window_size + 1):
                    for j in range(L_channel.shape[1] - window_size + 1):
                        window = L_channel[i:i+window_size, j:j+window_size]
                        window_A = lab_image[i:i+window_size, j:j+window_size, 1]
                        window_B = lab_image[i:i+window_size, j:j+window_size, 2]

                        features = self.feature_extractor(window, window_A, window_B)

                print(len(self.dicto))

    


    def colurize(self, target_image):
        window_size = 4
        target_image = cv2.resize(target_image, (64, 64))
        target_lab_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2LAB)

        target_L_channel = target_lab_image[:, :, 0]

        for i in range(target_L_channel.shape[0] - window_size + 1):
            for j in range(target_L_channel.shape[1] - window_size + 1):
                window = target_L_channel[i:i+window_size, j:j+window_size]
                window_A = target_lab_image[i:i+window_size, j:j+window_size, 1]
                window_B = target_lab_image[i:i+window_size, j:j+window_size, 2]

                features = self.feature_extractor_target_image(window, window_A, window_B)
                min_distance = 1000000000
                color_A, color_B = self.dicto[self.example]

                for key in self.dicto.keys():
                    distance = np.linalg.norm(features-key)
                    if distance < min_distance:
                        min_distance = distance
                        color_A, color_B = self.dicto[key]
                target_lab_image[i:i+window_size, j:j+window_size, 1] = color_A
                target_lab_image[i:i+window_size, j:j+window_size, 2] = color_B


        # Convert the image back to BGR color space
        target_image = cv2.cvtColor(target_lab_image, cv2.COLOR_LAB2BGR)
        
        return target_image



In [ ]:


target_image = cv2.imread('test.jpg')



colorizer = Colorize()

colorizer.train("C:/Users/azhar/Downloads/Similar mountains/Similar mountains")

colorized_image = colorizer.colurize(target_image)


plt.figure(figsize=(15, 5))

# plt.subplot(1, 3, 1)
# plt.title('Input Image')
# plt.imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
# plt.axis('off')

plt.subplot(1, 2, 1)
plt.title('Target Image')
plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Colorized Image')
plt.imshow(cv2.cvtColor(colorized_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.show()


In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title('Input Image')
plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY),cmap="gray")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title('Orignal Image')
plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title('Colorized Image')
plt.imshow(cv2.cvtColor(colorized_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.show()